In [9]:
from dotenv import load_dotenv
load_dotenv()

from langchain_community.document_loaders import PyPDFLoader

file_path = "pdfs/a_darwin-pipeline-duplication-process_2023.pdf"
loader = PyPDFLoader(file_path)

docs = loader.load()

print(len(docs))

53


In [4]:
print(docs[0].page_content[0:100])

Assessment Report 106
A
ssessment by supplementary environmental report 
D
arwin Pipeline Duplicatio


In [5]:
print(docs[0].metadata)

{'source': 'pdfs/a_darwin-pipeline-duplication-process_2023.pdf', 'page': 0}


In [13]:
import os

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o")

In [7]:
from langchain_chroma import Chroma
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=OpenAIEmbeddings())

retriever = vectorstore.as_retriever()

In [8]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

results = rag_chain.invoke({"input": "What was Darwin Pipeline Duplication ?"})

results

{'input': 'What was Darwin Pipeline Duplication ?',
 'context': [Document(metadata={'page': 41, 'source': 'pdfs/a_darwin-pipeline-duplication-process_2023.pdf'}, page_content='Draft Environmental Approval \nPa ge 1 of 11 \nPURSUANT TO SECTION 69 OF THE ENVIRONMENT PROTECTION ACT 2019 \nApproval number EP2022/022-001 \nApproval holder Santos NA Barossa Pty Ltd \nAustralian business number (ABN) 44 109 974 932 \nRegistered business address 60 Flinders Street, Adelaide SA 5000 \nAction Darwin Pipeline Duplication \nAction description \nConstruction, commissioning, operation and decommissioning of the Darwin Pipeline \nDuplication Project. The action is for a section of the Barossa gas pipeline that will \ntransport gas from the Timor Sea to the existing Darwin liquefied natural gas facility. The \napproved action includes an approximately 100 km section of pipeline in NT waters, a \nspoil disposal area, and a shore crossing at Wickham Point in the Greater Darwin Area. \nConstruction of th